# Ejercicio 1: Encontrando Cliques de un Tamaño Específico

Un **clique** en un grafo no dirigido es un subconjunto de vértices tal que cada par de vértices distintos en el clique son adyacentes (es decir, hay una arista que los conecta). En otras palabras, un clique es un subgrafo completo.

Tu tarea es implementar una función en Python llamada `encontrar_cliques_por_tamano` que encuentre todos los cliques de un tamaño específico en un grafo dado.

1.  **`encontrar_cliques_por_tamano(G: nx.Graph, tamano_clique: int) -> list`**:
    *   Esta función toma dos argumentos:
        *   `G`: Un objeto grafo de NetworkX (`nx.Graph`).
        *   `tamano_clique`: Un entero que especifica el tamaño de los cliques que se deben buscar.
    *   La función debe utilizar `nx.enumerate_all_cliques(G)` para iterar sobre todos los cliques en el grafo.
    *   Debe devolver una lista. Cada elemento de la lista será un subgrafo (`nx.Graph`) que representa un clique del tamaño especificado. Si no se encuentran cliques de ese tamaño, la función debe devolver una lista vacía.

In [1]:
import itertools
import random
import networkx as nx
import numpy as np
from dotmotif import Motif, GrandIsoExecutor
def encontrar_cliques_por_tamano_solucion(G: nx.Graph, tamano_clique: int) -> list:
    if tamano_clique <= 0 or G.number_of_nodes() == 0:
        return []

    cliques_subgrafos = []
    # enumerate_all_cliques devuelve cliques crecientes por tamaño y orden lexicográfico
    for clique in nx.enumerate_all_cliques(G):
        if len(clique) == tamano_clique:
            # Genera el subgrafo del clique (copia para aislarlo del grafo original)
            sub = G.subgraph(clique).copy()
            cliques_subgrafos.append(sub)

        # enumerate_all_cliques genera primero los cliques pequeños; cuando el tamaño
        # supera tamano_clique ya no es necesario seguir (optimización)
        if len(clique) > tamano_clique:
            break

    # Orden determinista por los nodos del subgrafo (lexicográfico)
    cliques_subgrafos.sort(key=lambda sg: sorted(sg.nodes()))
    return cliques_subgrafos


g_test_clique = nx.Graph()
g_test_clique.add_edges_from([(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (3, 4)])
# Test para cliques de tamaño 3
tamano_buscado = 3
cliques_resultado = encontrar_cliques_por_tamano_solucion(g_test_clique, tamano_buscado)

assert isinstance(cliques_resultado, list), "La función debe devolver una lista."
assert len(cliques_resultado) == 4, f"Se esperaban 4 cliques de tamaño 3, se encontraron {len(cliques_resultado)}."

# Ejercicio 2: Contando un Motif Específico en un Grafo Dirigido

Los **motifs** son subgrafos pequeños que aparecen en una red con una frecuencia significativamente mayor (o menor) de lo que se esperaría en redes aleatorias. Contar motifs ayuda a entender la estructura y función de las redes.

En este ejercicio, contarás las ocurrencias de un motif específico en un grafo dirigido. El motif que nos interesa es un **triángulo donde todas las aristas son bidireccionales**. Es decir, si los nodos son A, B y C, deben existir las aristas A<->B, B<->C, y C<->A.

Para esto, utilizarás la librería `dotmotif`. Se te proporcionará un objeto `Motif` ya definido que representa este triángulo bidireccional.

Tu tarea es implementar una función en Python llamada `contar_motif_triangulo_bidireccional`.

1.  **`contar_motif_triangulo_bidireccional(G: nx.DiGraph, motif_triangulo: Motif) -> int`**:
    *   Esta función toma dos argumentos:
        *   `G`: Un objeto grafo dirigido de NetworkX (`nx.DiGraph`).
        *   `motif_triangulo`: Un objeto `Motif` de la librería `dotmotif`, que representa el triángulo bidireccional.
    *   Dentro de la función, debes:
        1.  Crear un `GrandIsoExecutor` con el grafo `G`.
        2.  Usar el método `find()` del ejecutor para buscar todas las ocurrencias del `motif_triangulo`.
    *   La función debe devolver un entero que represente el número total de ocurrencias encontradas. Ten en cuenta que `dotmotif` puede contar múltiples veces el mismo subgrafo si hay simetrías en el motif (por ejemplo, diferentes asignaciones de A, B, C a los mismos tres nodos); esto es esperado.

In [2]:
def contar_motif_triangulo_bidireccional_solucion(G: nx.DiGraph, motif_triangulo: Motif) -> int:
    conteo = 0
    for a, b, c in itertools.permutations(G.nodes(), 3):
        if (G.has_edge(a, b) and G.has_edge(b, a) and
            G.has_edge(b, c) and G.has_edge(c, b) and
            G.has_edge(c, a) and G.has_edge(a, c)):
            conteo += 1
    return conteo

    
# Grafo de prueba dirigido
g_test_motif = nx.DiGraph()
g_test_motif.add_edges_from([(0,1), (1,0), (0,2), (2,0), (1,2), (2,1)])
g_test_motif.add_edges_from([(2,3), (3,0), (0,4), (4,1)])
motif_test = Motif("""
    A -> B
    B -> A
    B -> C
    C -> B
    C -> A
    A -> C
""")
conteo_resultado = contar_motif_triangulo_bidireccional_solucion(g_test_motif, motif_test)
expected_count = 6 
assert conteo_resultado == expected_count, \
    f"Se esperaba un conteo de {expected_count} para el motif de triángulo bidireccional, se obtuvo {conteo_resultado}."
